In [2]:
import os
import pandas as pd
import re
print(os.getcwd())

c:\Users\Joshualevy\Documents\ioCapture\01_Database_Construction\papers


In [10]:
fuzzy_reports_paths_lists = []
fuzzy_matched_csvs_lists = []
fuzzy_results_folders_path = '../../00_Scraping/econlit_scopus_matching_out/'
for current_folder,child_folders,current_folder_files in os.walk(fuzzy_results_folders_path):
    for file in current_folder_files:
        if re.search(r'_fuzzy_matching_report\.txt', file, re.I):
            new_report_path = current_folder + '/' + file
            fuzzy_reports_paths_lists.append(new_report_path)

        ### In the future this should change to accomodate the hand-matched articles as well. 
        if re.search(r'_fuzzy_matches\.csv', file, re.I):
            fuzzy_matches_path = current_folder + '/' + file
            fuzzy_matched_csvs_lists.append(fuzzy_matches_path)

# print(fuzzy_reports_paths_lists)
# print(fuzzy_matched_csvs_lists)


In [ ]:
fuzzy_reports_df = pd.DataFrame()

for path in fuzzy_reports_paths_lists:
    print(path)
    pub_code = re.search(r'([A-Z]{3})_fuzzy_matching_report\.txt', path)[1]
    print(pub_code)

    fuzzy_report_patterns_dict = {
        'scopus_obs' : r'Original ' + pub_code + r' Scopus-collected observations: (\d{3,5})',
        'econlit_obs' : r'Original ' + pub_code + r' EconLit-collected observations: (\d{3,5})',
        'naive_match_obs' : r'Number of ' + pub_code + r' naively-matched observations: (\d{3,5})',
        'fuzzy_match_obs' : r'Number of ' + pub_code + r' fuzzy-matched observations \d+ \((\d{2,5}) added\)',
        'remaining_scopus_obs' : r'Number of ' + pub_code + r' post-fuzzy-match Scopus-only remainder observations: (\d{1,5})',
        'remaining_econlit_obs' : r'Number of ' + pub_code + r' post-fuzzy-match EconLit-only remainder observations: (\d{1,5})',
    }


    temp_df = pd.DataFrame({
        'pub_code' : pub_code,
        'scopus_obs' : [None],
        'econlit_obs' : [None],
        'naive_match_obs' :  [None],
        'fuzzy_match_obs' : [None],
        'remaining_scopus_obs' : [None],
        'remaining_econlit_obs' : [None],

    })

    fuzzy_report_file = open(path, 'r', encoding='utf-8')
    fuzzy_report_lines = fuzzy_report_file.read()

    for column in fuzzy_report_patterns_dict.keys():
        pattern = fuzzy_report_patterns_dict.get(column)
        patterns_value = re.search(pattern, fuzzy_report_lines)[1]
        print('{}: {}'.format(column, patterns_value))
        temp_df.loc[0, column] = patterns_value

    fuzzy_reports_df = pd.concat([fuzzy_reports_df, temp_df], ignore_index=True)


fuzzy_reports_df.to_csv('fuzzy_match_results.csv', encoding='utf-8', index=False)


In [19]:
fuzzy_matched_papers_df = pd.DataFrame()

for file in fuzzy_matched_csvs_lists:
    temp_df = pd.read_csv(file)

    fuzzy_matched_papers_df = pd.concat([fuzzy_matched_papers_df, temp_df])
    print(len(fuzzy_matched_papers_df))

fuzzy_matched_papers_df.columns
# fuzzy_matched_papers_df = fuzzy_matched_papers_df.drop(columns='')
fuzzy_matched_papers_df.to_csv('fuzzy_matched_papers_df.csv', index=False, encoding='utf-8')


4803
5482
7049
7756
10370
11311
12035
12749
15173
15987
17279
18554
19965
21928
23097
